### Impor library

In [1]:
import requests as r
from bs4 import BeautifulSoup as bs
from urllib.request import Request, urlopen
import pandas as pd
from time import time, sleep
import os

### Inisiasi scrapper google news

In [2]:
class GOOGLENEWS_SCRAPPER(object):
    def __init__(self, QUERY, SITE, MIN_MONTH, MIN_DATE, MIN_YEAR, MAX_MONTH, MAX_DATE, MAX_YEAR):
        self.query = QUERY
        self.site = SITE
        self.min_month = MIN_MONTH
        self.min_date = MIN_DATE
        self.min_year = MIN_YEAR
        self.max_month = MAX_MONTH
        self.max_date = MAX_DATE
        self.max_year = MAX_YEAR
        
    def append_news(self, news, start_page):
        news_temp = []
        count = 0
        URL = "https://www.google.com/search?q={query}+site:{site}&safe=strict&client=firefox-b-d&tbs=sbd:1,nsd:1,qdr:y&tbm=nws&ei=sH_rX9yDNKLC3LUPguipiAI&start={start_page}&sa=N&filter=0&ved=0ahUKEwjcm6Hp8vPtAhUiIbcAHQJ0CiE4FBDy0wMIhwE&biw=1525&bih=730&dpr=0.9" # setahun terakhir,urutan tanggal, duplikat
        req = Request(URL.format(query = self.query, 
                                 site = self.site, 
                                 min_month=self.min_month, min_date=self.min_date, min_year=self.min_year, 
                                 max_month=self.max_month, max_date=self.max_date, max_year=self.max_year,
                                 start_page=start_page
                                ),
                      headers={'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5)\
            AppleWebKit/537.36 (KHTML, like Gecko) Cafari/537.36'})
        webpage = urlopen(req).read()
        with r.Session() as c:
            soup = bs(webpage, 'html5lib')
            for item in soup.find_all('div',attrs={'class':'kCrYT'}):
                if count%2==0:
                    title = item.select('div.BNeawe.vvjwJb.AP7Wnd')[0].text
                    site = item.select('div.BNeawe.UPmit.AP7Wnd')[0].text
                    link = item.find('a',attrs={'href':True})['href'].split('/url?q=')[1]
                else:
                    time = item.select('span.r0bn4c.rQMQod')[0].text
                    text = item.select('div.BNeawe.s3v9rd.AP7Wnd')[0].text.split(' · ')[1]
                    news.append([link,time,site,title,text])
                    news_temp.append([link,time,site,title,text])
                    link = "" ; title = ""; site = ""; time = ""; text = ""
                count+=1
        return news, news_temp
    
    def scrap(self):
        self.news = []
        self.news_temp = []
        
        start_page = 0
        append_again = True
        i = 0;
        
        while append_again:
            self.news, self.news_temp= self.append_news(self.news, start_page)
            i+=1; print('collecting news: page {}'.format(i))
            sleep(5)
            if len(self.news_temp)<10:
                append_again = False
            else:
                start_page +=10
        print('Result:{} news items.'.format(i*10+len(self.news_temp)))
        return self.news
    
    def scrap_toPandas(self):
        self.data = pd.DataFrame(self.news, columns = ['Link','Time','Site','Title','Text'])
        return self.data
        
    def scrap_save(self):
        FOLDER_DIR = os.path.join(os.getcwd(), "googlenews")
        if not os.path.exists(FOLDER_DIR):
            os.mkdir(FOLDER_DIR)
        FILENAME = "{}_{}_{}-{}-{}_{}-{}-{}.csv".format(self.query, self.site, self.min_month, self.min_date, self.min_year, self.max_month, self.max_date, self.max_year)
        FILE_DIR = os.path.join(FOLDER_DIR, FILENAME)
        self.data.to_csv(FILE_DIR)
        print('Data saved to:{}'.format(FILE_DIR))

### Menjalankan scrapper google news

In [6]:
# Set Values
QUERY = "kecelakaan"
SITE = ["radargresik.jawapos.com"]
MIN_MONTH = 1; MIN_DATE = 1; MIN_YEAR = 2020; MAX_MONTH = 12; MAX_DATE = 31; MAX_YEAR = 2020
n = 2; wait = 1
for i in range(len(SITE)):
    print('Scrap:{}'.format(SITE[i]))
    scrapper = GOOGLENEWS_SCRAPPER(QUERY, SITE[i], MIN_MONTH, MIN_DATE, MIN_YEAR, MAX_MONTH, MAX_DATE, MAX_YEAR)
    success = False
    while not success:
        try:
            news = scrapper.scrap()
            success = True
            scrapper.scrap_toPandas()
            scrapper.scrap_save()
        except Exception as e:
            wait = wait * n;
            print('[%s] Error! Waiting %s secs and re-trying...' % (e, wait))
            sleep(wait)

Scrap:radargresik.jawapos.com
collecting news: page 1
collecting news: page 2
collecting news: page 3
collecting news: page 4
collecting news: page 5
collecting news: page 6
collecting news: page 7
collecting news: page 8
collecting news: page 9
collecting news: page 10
collecting news: page 11
Result:115 news items.
Data saved to:C:\Users\Dwi Nissa\Skripsi\CARI METODE\SCRAPPING\googlenews\kecelakaan_radargresik.jawapos.com_1-1-2020_12-30-2020.csv
